In [8]:
import pandas as pd
import numpy as np
import os, sys, re
from StringIO import StringIO
import itertools

In [9]:
sys.path.append("./static/python/")
# sys.path.append("./../metaprot/sql/")

In [10]:
import db_config

In [11]:
connection = db_config.Connect(echo=True, testing=False, do_logging=False)
protein_ans_list = ["P35997", "P38711", "P35997", "P38711", "B2RID1", "belk_c_455_5138"]
# protein_ans_list = ["B2RID1"]

In [12]:
# stmt = """SELECT protein_2_function.an, protein_2_function.function
# FROM protein_2_function
# INNER JOIN functions ON protein_2_function.function=functions.an
# INNER JOIN ontologies ON ontologies.child=functions.an
# WHERE protein_2_function.an IN('P35997')
# AND functions.type='GO'
# ;""" #AND ontologies.parent='GO:0003674'
# session = connection.get_session()
# result = session.execute(stmt).fetchall()
# session.close()

In [13]:
def parse_result_child_parent(result):
    return sorted(set([item for sublist in result for item in sublist]))

In [14]:
def parse_result_an_child_parent(result):
    an_2_functions_dict = {}
    for res in result:
        an = res[0]
        function = res[1:]
        if not an in an_2_functions_dict:
            for func in function:
                an_2_functions_dict[an] = {func}
        else:
            for func in function:
                an_2_functions_dict[an].update([func])
    return an_2_functions_dict

In [15]:
import userinput
reload(userinput)
import query, db_config
reload(query)
reload(db_config)

<module 'db_config' from './static/python/db_config.pyc'>

In [18]:
# limit_2_parent invokes backtracking=False
an = "P35997"
d = query.get_association_dict(connection, protein_ans_list=[an], 
                           function_type="KEGG", limit_2_parent=None, 
                           basic_or_slim="basic", backtracking=False)
print "#"*80
print len(d[an])
x = d[an]
d[an]

2016-09-27 18:06:43,439 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-09-27 18:06:43,440 INFO sqlalchemy.engine.base.Engine SELECT protein_2_function.an, protein_2_function.function
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='KEGG'
;
2016-09-27 18:06:43,441 INFO sqlalchemy.engine.base.Engine {}
2016-09-27 18:06:43,443 INFO sqlalchemy.engine.base.Engine ROLLBACK
2016-09-27 18:06:43,444 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-09-27 18:06:43,444 INFO sqlalchemy.engine.base.Engine SELECT protein_2_og.an, og_2_function.function
FROM protein_2_og
INNER JOIN og_2_function ON protein_2_og.og=og_2_function.og
INNER JOIN functions ON og_2_function.function=functions.an
WHERE protein_2_og.an IN('P35997')
AND functions.type='KEGG'
;
2016-09-27 18:06:43,445 INFO sqlalchemy.engine.base.Engine {}
2016-09-27 18:06:43,449 INFO sqlalchemy.engine.base.Engine ROLLBACK
#####

{u'KEGG:03010'}

In [ ]:
kegg_path = "sce03010"
"KEGG:{}".format(kegg_path[3:])

In [ ]:
sql_stmt = """
SELECT protein_2_function.an, protein_2_function.function
FROM protein_2_function
WHERE protein_2_function.an IN('P35997')
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
print "#"*80
result

In [ ]:
# GO:0003735 # MF
# GO:0006412 # BP

In [ ]:
# temp
sql_stmt = """SELECT *
FROM functions
WHERE functions.name='Technical term'
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
print "#"*80
print result

In [ ]:
# temp
sql_stmt = """SELECT *
FROM ontologies
WHERE ontologies.child='UPK:0002'
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
print "#"*80
print result

In [ ]:
### subquery (SELECT * FROM ontologies WHERE ontologies.parent='GO:0008150')
sql_stmt = """SELECT ontologies.child, ontologies.parent
FROM ontologies
WHERE ontologies.parent='GO:0008150'
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
# an_2_functions_dict = parse_result_an_child_parent(result)
print "#"*80
print result[:10]
print len(result)
# print len(an_2_functions_dict["P35997"])
# an_2_functions_dict["P35997"]

In [ ]:
### subquery, takes forever, make it shorter
sql_stmt = """SELECT protein_2_function.an, ontologies.child, ontologies.parent 
FROM ontologies
INNER JOIN protein_2_function ON ontologies.child=protein_2_function.function
SELECT protein_2_function.function
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='GO'
AND ontologies.parent = 'GO:0008150')
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
# an_2_functions_dict = parse_result_an_child_parent(result)
print "#"*80
print result
# print len(an_2_functions_dict["P35997"])
# an_2_functions_dict["P35997"]

In [ ]:
### subquery, takes forever
sql_stmt = """SELECT protein_2_function.an, ontologies.child, ontologies.parent 
FROM ontologies
INNER JOIN protein_2_function ON ontologies.child=protein_2_function.function
WHERE ontologies.child IN(
SELECT protein_2_function.function
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='GO'
AND ontologies.parent = 'GO:0008150')
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
# an_2_functions_dict = parse_result_an_child_parent(result)
print "#"*80
print result
# print len(an_2_functions_dict["P35997"])
# an_2_functions_dict["P35997"]

In [ ]:
sql_stmt = """SELECT ontologies.child, ontologies.parent FROM ontologies WHERE ontologies.child='GO:0006412'
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
print "#"*80
print len(result)
result[:10]

In [ ]:
sql_stmt = """SELECT protein_2_function.an, ontologies.child, ontologies.parent
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
INNER JOIN go_2_slim ON go_2_slim.an=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='GO'
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
print "#"*80
result

In [ ]:
list(an_2_functions_dict[u'P35997']).count('GO:0008150')

In [ ]:
counter = 0
for res in result:
    if 'P35997' in list(res):
        counter += 1
print counter

In [ ]:
u'P35997' in (u'P35997', u'GO:0000028', u'GO:0065003')

In [ ]:
# first select
sql_stmt = """SELECT protein_2_function.an, ontologies.child, ontologies.parent
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
INNER JOIN go_2_slim ON go_2_slim.an=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='GO'
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
an_2_functions_dict = parse_result_an_child_parent(result)
print "#"*80
len(an_2_functions_dict["P35997"])

In [ ]:
# second select
sql_stmt = """SELECT ontologies_1.child, ontologies_1.parent FROM ontologies AS ontologies_1
WHERE ontologies_1.child IN({})
AND ontologies_1.parent = 'GO:0008150';""".format(str([str(ele) for ele in an_2_functions_dict["P35997"]])[1:-1])
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
print "#"*80
result
# result = parse_result_child_parent(result)
# print len(result)

In [ ]:
### original faulty version
SELECT protein_2_function.an, ontologies.child, ontologies.parent
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
INNER JOIN go_2_slim ON go_2_slim.an=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='GO'
AND ontologies.parent='GO:0008150'
;
### first select
SELECT protein_2_function.an, ontologies.child, ontologies.parent
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
INNER JOIN go_2_slim ON go_2_slim.an=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='GO'
;
### second select
SELECT ontologies.child, ontologies.parent FROM ontologies
WHERE ontologies.child IN('GO:0044249', 'GO:0034641', 'GO:0006807', 'GO:0003674', 'GO:0034645', 'GO:1901576', 'GO:0044260', 'GO:0043043', 'GO:0005198', 'GO:0003735', 'GO:0071704', 'GO:0044267', 'GO:0009987', 'GO:0009058', 'GO:0009059', 'GO:0008150', 'GO:0008152', 'GO:0044271', 'GO:0043604', 'GO:0044238', 'GO:1901564', 'GO:0043603', 'GO:1901566', 'GO:0019538', 'GO:0044237', 'GO:0043170', 'GO:0006518', 'GO:0006412')
AND ontologies.parent = 'GO:0008150'
;

In [ ]:
SELECT protein_2_function.an, ontologies.child, ontologies.parent
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
INNER JOIN go_2_slim ON go_2_slim.an=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='GO'
AND ontologies.parent = 'GO:0008150'
;

In [ ]:
sql_stmt = """SELECT protein_2_function.an, ontologies.child, ontologies.parent
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
INNER JOIN go_2_slim ON go_2_slim.an=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='GO'
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
len(result)

In [ ]:
sql_stmt = """SELECT protein_2_og.an, og_2_function.function
FROM protein_2_og
INNER JOIN og_2_function ON protein_2_og.og=og_2_function.og
INNER JOIN functions ON og_2_function.function=functions.an
INNER JOIN go_2_slim ON go_2_slim.an=functions.an
WHERE protein_2_og.an IN('P35997')
AND functions.type='GO'
;"""
session = connection.get_session()
result = session.execute(sql_stmt).fetchall()
session.close()
len(result)

In [ ]:
d

In [ ]:
from subprocess import call

In [ ]:
l = ['GO:0000028', 'GO:0000462','GO:0002181','GO:0003735','GO:0006412','GO:0022627','GO:0046872']


In [ ]:
# all directly annotated GO terms for "P35997"
x = """3782218	P35997	GO:0000028
3782219	P35997	GO:0000462
3782220	P35997	GO:0002181
3782221	P35997	GO:0003735
3782222	P35997	GO:0006412
3782223	P35997	GO:0022627
3782224	P35997	GO:0046872"""
go_list = []
for ele in x.split("\n"):
    go_list.append(ele.split("\t")[-1])
print len(go_list)
go_list    

In [ ]:
x = """290220159	P35997	UPK:0488
290220160	P35997	UPK:0963
290220161	P35997	UPK:0002
290220162	P35997	UPK:1185
290220163	P35997	UPK:0863
290220164	P35997	UPK:0862
290220165	P35997	UPK:0479
290220166	P35997	UPK:0687
290220167	P35997	UPK:0689
290220168	P35997	UPK:0181"""
go_list = []
for ele in x.split("\n"):
    go_list.append(ele.split("\t")[-1])
print len(go_list)
go_list

In [ ]:
print go_list

In [ ]:
upkTerm_2_functionAN_dict = {u'Biological process': u'UPK:9999',
                             u'Cellular component': u'UPK:9998',
                             u'Coding sequence diversity': u'UPK:9997',
                             u'Developmental stage': u'UPK:9996',
                             u'Disease': u'UPK:9995',
                             u'Domain': u'UPK:9994',
                             u'Ligand': u'UPK:9993',
                             u'Molecular function': u'UPK:9992',
                             u'Technical term': u'UPK:9990'}
len(upkTerm_2_functionAN_dict)

In [ ]:
x = sorted(x)
x = sorted([str(ele) for ele in x])
x == go_all

In [ ]:
stmt = """SELECT ontologies.child FROM ontologies
WHERE ontologies.child IN({})
AND ontologies.parent = 'GO:0008150';""".format(str(go_list)[1:-1])
session = connection.get_session()
result = session.execute(stmt).fetchall()
session.close()
result

In [ ]:
stmt = """SELECT ontologies.child AS ont_1 FROM ontologies
WHERE ontologies.child IN({})
AND ontologies.parent = 'GO:0008150';""".format(str(go_list)[1:-1])
session = connection.get_session()
result = session.execute(stmt).fetchall()
session.close()
result

In [ ]:
# all parents of go_list
# go_list = ['GO:0000028', 'GO:0000462', 'GO:0002181', 'GO:0006412', ]
stmt = """SELECT ontologies.child, ontologies.parent FROM ontologies AS ont_1
INNER JOIN ontologies ON 
WHERE ontologies.child IN({});""".format(str(go_list)[1:-1])
#AND ontologies.parent IN('GO:0005575', 'GO:0008150', 'GO:0003674')
# ontologies.child, ontologies.parent FROM ontologies
# AND ontologies.parent = 'GO:0008150'
# GO:0005575
# GO:0008150
# GO:0003674
'GO:0005575', 'GO:0008150', 'GO:0003674'
session = connection.get_session()
result = session.execute(stmt).fetchall()
session.close()
# go_all = []
# for res in result:
#     go_all += list(res)
# go_all = sorted(set(go_all))
# print len(go_all)
# go_all
result

In [ ]:
s = StringIO()

In [ ]:
f = u'BUBU;BUBU2;BUBU3\r\nAUBU;AUBU2;AUBU3\r\nP40545\r\nP09201\r\nP14066\r\nP25040\r\nP20050\r\nP40327\r\nQ01939\r\nQ01939'
b = u'BUBU\t\r\nBUBU\t\r\nP00359\t11,07751325\r\nP00925\t10,96978388\r\nP02994\t10,76849751\r\nP06169\t10,74716312\r\nP00560\t10,74614359\r\nP00950\t10,74149039\r\nP00549\t10,70474233\r\nP00330\t10,6690378\r\nAIAIAI\r\nUUUU;IIII'

In [ ]:
# zip(f.split("\r\n"), b.split("\r\n"))

In [ ]:
s = StringIO()
s.write('foreground\tbackground\tintensity\r')
for a, b in itertools.izip_longest(f.split("\r\n"), b.split("\r\n"), fillvalue="\t"):
    s.write(a + "\t" + b.strip() + "\r")

In [ ]:
s.tell()

In [ ]:
s.seek(0)

In [ ]:
s.tell()

In [ ]:
set(["all_GO", "BP", "CP", "MF"])

In [ ]:
{"BP", "CP", "MF"}

In [ ]:
s.read()

In [ ]:
s.split("\r")

In [ ]:
x = 'intensity\tbackground\tforeground\r\tBUBU\t"BUBU;BUBU2;BUBU3"\r\tBUBU\tAUBU;AUBU2;AUBU3\r11,07751325\tP00359\tP40545\r10,96978388\tP00925\tP09201\r10,76849751\tP02994\tP14066\r10,74716312\tP06169\tP25040\r10,74614359\tP00560\tP20050\r10,74149039\tP00950\tP40327\r10,70474233\tP00549\tQ01939\r10,6690378\tP00330\tQ01939\r10,64206863\tP05743\tP38142\r10,64061061\t"P02407;P14127"\tQ01939\r10,63619693\t"P0C2H6;P0C2H7"\tP53549\r10,6231769\t"P35997;P38711"\tP53549\r10,61589747\tP29453\tP39959\r10,59787062\tO14455\tP38787\r10,59595893\tP04456\tP38787\r10,5819724\tP14540\tP12684\r10,58108453\t"P0CX84;P0CX85"\tP12684\r10,58103895\tP05755\tP38122\r10,56810781\t"P0CX31;P0CX32"\tP38122\r10,54364595\t"P0CX55;P0CX56"\tP0CX47\r10,5379072\tP38061'

In [ ]:
x.split("\r")

In [ ]:
s.write("bubu")

In [ ]:
s.tell()

In [ ]:
fn = r"/Users/dblyon/CloudStation/CPR/BTW_GO/UserInput.txt"
fn = r"/Users/dbl/modules/cpr/agotool/static/data/exampledata/exampledata_yeast.txt"
# fn = r"/Users/dblyon/modules/cpr/agotool/static/data/exampledata/refactorUserinput.txt"
df = pd.read_csv(fn, sep='\t', decimal=",")#, converters={"background_int": float, "background_an": str, "foreground_an": str})

In [ ]:
# fh = open(fn, "r").read()
ui = userinput.Userinput(fn=fn, foreground_string=None, background_string=None,
col_foreground='foreground', col_background='background', col_intensity='intensity',
num_bins=100, decimal='.', method="abundance_correction")

In [ ]:
ui.background

In [ ]:
cond = ui.foreground["intensity"] > -1
bins = pd.cut(ui.foreground.loc[cond, "intensity"], bins=100, retbins=True)[1]
bins = [-1] + list(bins)

In [ ]:
ui.foreground

In [ ]:
for x in ui.iter_bins():
#     print wf, bins
#     print bg, fg
#     print ans, cf
    print x
    print

In [ ]:
mbps = 30.0
gb = mbps / 1000.0
min_ = gb * 60
hour = min_ * 60
print min_
print hour
tb = 2.0
(tb * 1000) / hour

In [ ]:
bins = pd.cut(ui.foreground["intensity"], bins=100, retbins=True)[1]
groups_fg = ui.foreground.groupby(pd.cut(ui.foreground["intensity"], bins=bins))
groups_bg = ui.background.groupby(pd.cut(ui.background["intensity"], bins=bins))

In [ ]:
# ui.background[ui.background["intensity"] <= 0]

In [ ]:
for group_fg, group_bg in zip(groups_fg, groups_bg):
    bins_in_group = group[0]
    pg_fg = group_fg[1]["foreground"]
    pg_bg = group_bg[1]["background"]
    print pg_fg, len(pg_fg)
    print pg_bg, len(pg_bg)
    print 
    break

In [ ]:
# for group in groups:
# #     bins_in_group = group[0]
# #     pg_in_group = group[1]["foreground"]
# #     print bins_in_group
# #     print pg_in_group
#     print group
#     print 

In [ ]:
hist, bins = np.histogram(ui.foreground["intensity"], bins=100)

In [ ]:
bins # (-1.0121, -0.879]

In [ ]:
hist

In [ ]:
10.71518785 -  10.83596299

In [ ]:
9.62821166 -   9.74898679

In [ ]:
df.loc[0, "background_int"]

In [ ]:
# np.histogram(df["background_int"], bins=10)
np.histogram(df.loc[pd.notnull(df["background_int"]), "background_int"], bins=10)

In [ ]:
ui.background_df

In [ ]:
# import math

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

In [ ]:
# arr = list(np.arange(10))
# arr += [2.3]*8
# hist, bin_edges = np.histogram(arr, bins=10, density=False)
# plt.hist(arr, bins=bin_edges)  # plt.hist passes it's arguments to np.histogram
# plt.show()

In [ ]:
# good stuff

In [ ]:
# foreground proteinGroups with intensities
intensity_foreground = []
for proteinGroup in ui.foreground_ser:
    an_first_in_proteinGroup = proteinGroup.split(";")[0]
    intensity_foreground.append(ui.an_2_intensity_dict[an_first_in_proteinGroup])
#     try:      
#         intensity_foreground.append(ui.an_2_intensity_dict[an_first_in_proteinGroup])
#     except KeyError:
#         # can't occur, since defaultdict
#         raise StopIteration

In [ ]:
ui.foreground_df = pd.DataFrame(data=intensity_foreground, columns=["Int"])
ui.foreground_df["AN"] = ui.foreground_ser

In [ ]:
df = ui.foreground_df
df

In [ ]:
1.5*30

In [ ]:
# create bins
# which proteinGroups from background are within bin

In [ ]:
num_bins = 100
bins = np.linspace(df["Int"].min(), df["Int"].max(), num_bins)
groups = df.groupby(pd.cut(df["Int"], bins))

In [ ]:
# for group in groups:
#     print group
#     print